In [13]:
#! pip install xgboost

In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
#import one hot encoder
from sklearn.preprocessing import OneHotEncoder
# suppress warnings
import warnings
warnings.filterwarnings("ignore")
import xgboost
from math import log
import os
import pickle
import sys
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [15]:
sys.path.append('../Library')

from data_prep import PreProcessor1

import library_leo as leo

# import the data

In [16]:
from sklearn.model_selection import train_test_split
# get the data
train_path = "../Data/train_data.csv"
test_path = "../Data/test_data.csv"
# Replace 'Column29' with the actual column name that has mixed types
dtype = {'Column29': 'str'}

train = pd.read_csv(train_path, index_col='Claim Identifier',
                    dtype=dtype, low_memory=False)
test = pd.read_csv(test_path, index_col='Claim Identifier',
                   dtype=dtype, low_memory=False)
train = train.dropna(subset=['Claim Injury Type'])
y = train['Claim Injury Type']
train = train.dropna(subset=['Claim Injury Type'])
y = train['Claim Injury Type']
y_str = y.str[:1]
y_int = y_str.astype(int)
X = train.drop(columns=['Claim Injury Type'])
# partition the data X, y and y_2bin
X_train, X_valid, y_hot_train,y_hot_valid = train_test_split(
    X, y_int, test_size=0.2, random_state=0)



# prepare the data

In [17]:
os.chdir('../Models')

pr = pickle.load(open('prepProcessor41.pkl', 'rb'))

In [18]:
pr.update_status('train')
print("------------updates---------------------")
pr.update_casted_cols()
print("------------set_casting---------------------")
pr.set_castings(X_train)
print("------------pipeline---------------------")
df_train = pr.cast_pipeline(X_train)
print("df after cast_pipeline:", df_train.shape)
print("------------fillna---------------------")
pr.update_fillna_list(df_train)   
df_train = pr.fillna_pipeline(df_train)
print("df after fillna_pipeline:", df_train.shape)
print("------------transformation---------------------")
pr.set_transformations(df_train)
df_train = pr.transformation_pipeline(df_train)
print("df after transformation_pipeline:", df_train.shape)
df_train= pr.scaling_pipeline(df_train)
df_train = df_train.drop(columns=['Agreement Reached', 'WCB Decision'])
df_train.info() 

pr.update_status('valid')
print("------------updates---------------------")
pr.update_casted_cols()
print("------------set_casting---------------------")
pr.set_castings(X_valid)
print("------------pipeline---------------------")
df_valid = pr.cast_pipeline(X_valid)
print("df after cast_pipeline:", df_valid.shape)
print("------------fillna---------------------")
pr.update_fillna_list(df_valid)   
df_valid = pr.fillna_pipeline(df_valid)
print("df after fillna_pipeline:", df_valid.shape)
print("------------transformation---------------------")
pr.set_transformations(df_valid)
df_valid = pr.transformation_pipeline(df_valid)
print("df after transformation_pipeline:", df_valid.shape)
df_valid = pr.scaling_pipeline(df_valid)
df_valid = df_valid.drop(columns=['Agreement Reached', 'WCB Decision'])

pr.update_status('test')
print("------------updates---------------------")
pr.update_casted_cols()
print("------------set_casting---------------------")
pr.set_castings(test)
print("------------pipeline---------------------")
df_test = pr.cast_pipeline(test)
print("df after cast_pipeline:", df_test.shape)
print("------------fillna---------------------")
pr.update_fillna_list(df_test)   
df_test = pr.fillna_pipeline(df_test)
print("df after fillna_pipeline:", df_test.shape)
print("------------transformation---------------------")
pr.set_transformations(df_test)
df_test = pr.transformation_pipeline(df_test)
print("df after transformation_pipeline:", df_test.shape)
df_test = pr.scaling_pipeline(df_test)
df_test.info()




------------updates---------------------
------------set_casting---------------------
Columns that are not casted:
-WCB Decision: object
------------pipeline---------------------
Feature Carrier Type is already in scaling list
Feature Zip Code is already in scaling list
'Industry Code Description'
-Column Industry Code Description not found
"['Industry Code Description'] not found in axis"
-Column Industry Code Description not found
"['OIICS Nature of Injury Description'] not found in axis"
-Column OIICS Nature of Injury Description not found
"['WCIO Cause of Injury Description'] not found in axis"
-Column WCIO Cause of Injury Description not found
"['WCIO Nature of Injury Description'] not found in axis"
-Column WCIO Nature of Injury Description not found
"['WCIO Part Of Body Description'] not found in axis"
-Column WCIO Part Of Body Description not found
'Industry Code Description'
-Column Industry Code Description not found
"['Industry Code Description'] not found in axis"
-Column I

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459220 entries, 5877274 to 5803696
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Alternative Dispute Resolution  459220 non-null  int64  
 1   Attorney/Representative         459220 non-null  int64  
 2   Carrier Name                    459220 non-null  string 
 3   Carrier Type                    459220 non-null  string 
 4   County of Injury                459220 non-null  string 
 5   COVID-19 Indicator              459220 non-null  int64  
 6   District Name                   459220 non-null  object 
 7   Gender                          459220 non-null  string 
 8   Medical Fee Region              459220 non-null  string 
 9   Zip Code                        459220 non-null  Int64  
 10  First Hearing Month             459220 non-null  Int64  
 11  First Hearing Year              459220 non-null  Int64  
 12  Gender_F      

In [20]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114806 entries, 6143035 to 6081600
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Alternative Dispute Resolution  114806 non-null  int64  
 1   Attorney/Representative         114806 non-null  int64  
 2   Carrier Name                    114806 non-null  string 
 3   Carrier Type                    114806 non-null  string 
 4   County of Injury                114806 non-null  string 
 5   COVID-19 Indicator              114806 non-null  int64  
 6   District Name                   114806 non-null  object 
 7   Gender                          114806 non-null  string 
 8   Medical Fee Region              114806 non-null  string 
 9   Zip Code                        114806 non-null  Int64  
 10  First Hearing Month             114806 non-null  Int64  
 11  First Hearing Year              114806 non-null  Int64  
 12  Gender_F      

In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387975 entries, 6165911 to 6553594
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Alternative Dispute Resolution  387975 non-null  int64  
 1   Attorney/Representative         387975 non-null  int64  
 2   Carrier Name                    387975 non-null  string 
 3   Carrier Type                    387975 non-null  string 
 4   County of Injury                387975 non-null  string 
 5   COVID-19 Indicator              387975 non-null  int64  
 6   District Name                   387975 non-null  object 
 7   Gender                          387975 non-null  string 
 8   Medical Fee Region              387975 non-null  string 
 9   Zip Code                        387975 non-null  Int64  
 10  First Hearing Month             387975 non-null  Int64  
 11  First Hearing Year              387975 non-null  Int64  
 12  Gender_F      

# train data

In [23]:
folder_path = '../Models'

Train_predictions = []
Valid_predictions = []
Test_predictions = []

label_predictions = []

feature_used = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        print(f"File: {filename}")
        if filename.__contains__('final'):
    
            try:
                with open(file_path, 'rb') as f:
                    loaded_list = pickle.load(f)
                    loaded_object = pickle.load(f)
            except Exception as e:
                print(e)
                continue
            print(f"----------{filename}----------------")
            name, _ = filename.split('.')
            pred = loaded_object.predict(df_train[loaded_list])
            Train_predictions.append((name, pred))
            pred = loaded_object.predict(df_valid[loaded_list])
            Valid_predictions.append((name, pred))
            pred = loaded_object.predict(df_test[loaded_list])
            Test_predictions.append((name, pred))
            
            label_predictions.append(file_path.removeprefix(folder_path + '/')[0])
            
            feature_used.extend(loaded_list)
            feature_used = list(set(feature_used))
    elif os.path.isdir(file_path):
        print(f"Directory: {filename}")
        
    

File: 7LogisticRegression_final.sav
----------7LogisticRegression_final.sav----------------
File: 6MLPClassifier_1.sav
File: 8LogisticRegression_1.sav
File: 7LogisticRegression_1.sav
File: 8LogisticRegression_GridSearch_final.sav
----------8LogisticRegression_GridSearch_final.sav----------------
File: 8MLPClassifier_1.sav
File: 4_DecisionTreeClassifier_final.sav
----------4_DecisionTreeClassifier_final.sav----------------
File: 8_DecisionTreeClassifier_1.sav
File: 7XGBClassifier_1.sav
File: 6_DecisionTreeClassifier_1.sav
File: 6XGBClassifier_1.sav
File: 7_DecisionTreeClassifier_1.sav
File: 8XGBClassifier_1.sav
File: 8LogisticRegression_GridSearch_final_2.sav
----------8LogisticRegression_GridSearch_final_2.sav----------------
File: prepProcessor41.pkl
File: 6LogisticRegression_1.sav
File: 8LGBMClassifier_1.sav
File: 6RandomForestClassifier_1.sav
File: 7final_model.pkl
Ran out of input
File: 7LGBMClassifier_1.sav
File: 8RandomForestClassifier_1.sav
File: 6LGBMClassifier_1.sav
File: prep

In [24]:
label_predictions

['7', '8', '4', '8']

In [45]:
pred_df_train = pd.DataFrame([x[1] for x in Train_predictions]).T
pred_df_train.index = df_train.index
pred_df_train_numbered  = pred_df_train.copy()

for i in range(len(label_predictions)):
    pred_df_train_numbered.loc[:,i] = pred_df_train_numbered.loc[:,i].map({1:int(label_predictions[i]), 0: int(0)})


pred_df_valid = pd.DataFrame([x[1] for x in Valid_predictions]).T
pred_df_valid.index = df_valid.index
valid_df_train_numbered  = pred_df_valid.copy()

for i in range(len(label_predictions)):
    valid_df_train_numbered.loc[:,i] = valid_df_train_numbered.loc[:,i].map({1:int(label_predictions[i]), 0: int(0)})

pred_df_test = pd.DataFrame([x[1] for x in Test_predictions]).T
pred_df_test.index = df_test.index
pred_df_test_numbered  = pred_df_test.copy()

for i in range(len(label_predictions)):
    pred_df_test_numbered.loc[:,i] = pred_df_test_numbered.loc[:,i].map({1:int(label_predictions[i]), 0: int(0)})

# representation of the new space

In [ ]:
# tsne 

tsne = TSNE(n_components=2, random_state=0)
tsne_obj=tsne.fit_transform(pred_df_train)
pd.DataFrame(tsne_obj).plot.scatter(x=0, y=1)

# voting classifier 1

In [26]:
x = valid_df_train_numbered.T.copy()
# map the values of 0 to nanin x
x = x.replace(0, np.nan)
result = []
for col in x.columns:
    #append the most frequent value in x[col] to result withot using mode
    try:
        result.append(x[col].value_counts().idxmax())
    except:
        result.append(0)

pred = pd.DataFrame(result, index = x.columns)     

# voting classifier 2

In [27]:
x2 = valid_df_train_numbered.T.copy()
result2 = []
for col in x2.columns:
    #append the most frequent value in x[col] to result withot using mode
    result2.append(x2[col].value_counts().idxmax())

pred2 = pd.DataFrame(result2, index = x2.columns)    

In [28]:
pred2.value_counts()

0  
0.0    91936
8.0    22865
7.0        5
Name: count, dtype: int64

# basic voting approach

In [41]:
y_train_reincoded = y_hot_train.map({1:0, 2:0, 3:0, 4:4, 5:0, 6:0, 7:7, 8:8})
y_train_reincoded.index = y_hot_train.index

y_valid_reincoded = y_hot_valid.map({1:0, 2:0, 3:0, 4:4, 5:0, 6:0, 7:7, 8:8})
y_valid_reincoded.index = y_hot_valid.index

In [33]:
pred = pred.astype(int)
pred2 = pred2.astype(int)

In [ ]:
print(len(y_train_reincoded))
print(len(pred))

from sklearn.metrics import accuracy_score, classification_report


accuracy = accuracy_score(y_train_reincoded, pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_train_reincoded, pred))




114806
114806
Accuracy: 0.49858021357768756
              precision    recall  f1-score   support

           0       0.96      0.58      0.73     84877
           4       0.66      0.26      0.37     29822
           7       0.00      0.00      0.00        11
           8       0.00      0.85      0.00        96

    accuracy                           0.50    114806
   macro avg       0.40      0.42      0.27    114806
weighted avg       0.88      0.50      0.63    114806



In [ ]:
accuracy = accuracy_score(y_train_reincoded, pred2)
print(f"Accuracy: {accuracy}")

print(classification_report(y_train_reincoded, pred2))

Accuracy: 0.6714980053307319
              precision    recall  f1-score   support

           0       0.84      0.91      0.87     84877
           4       0.00      0.00      0.00     29822
           7       0.00      0.00      0.00        11
           8       0.00      0.64      0.01        96

    accuracy                           0.67    114806
   macro avg       0.21      0.39      0.22    114806
weighted avg       0.62      0.67      0.64    114806



In [48]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(pred_df_train, y_train_reincoded)
pred = model.predict(pred_df_valid)

# import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# classification_report
print(classification_report(pred, y_valid_reincoded))

# import some models 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# instantiate the models
list_of_models = [RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier(), DecisionTreeClassifier()]

# fit the models
for model in list_of_models:
    model.fit(pred_df_train, y_hot_train)
    pred = model.predict(pred_df_valid)
    print(f"Model: {model}")
    print(f"Accuracy: \n{accuracy_score(pred, y_valid_reincoded)}")
    print(f"Classification Report: \n{classification_report(pred, y_valid_reincoded)}")
    print(f"Confusion Matrix: \n{confusion_matrix(pred, y_valid_reincoded)}")
    print("-------------------------------------------------\n\n")
    

              precision    recall  f1-score   support

           0       0.85      0.94      0.89     76844
           4       0.84      0.66      0.74     37962
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.85    114806
   macro avg       0.42      0.40      0.41    114806
weighted avg       0.85      0.85      0.84    114806

Model: RandomForestClassifier()
Accuracy: 
0.22549344110934968
Classification Report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           2       0.00      0.00      0.00     74899
           4       0.87      0.66      0.75     39324
           5       0.00      0.00      0.00       583
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.23    114806
   macro avg       0.14      0.11      0.12 